In [1]:
import folium
import pandas as pd
import json
import requests
import time

В файле admin_level_4.geojson хранятся данные для отрисовки контуров регионов 

In [2]:
with open('admin_level_4.geojson', encoding='utf-8') as json_file:
    data = json.load(json_file)

Извлечем инф-ию о том, какой номер в данном файле соответствует каждому региону

In [ ]:
region_id = []
reg_list = data['features'][1:]
for el in reg_list:
    region_id.append((el['name'], el['id'])) 

In [ ]:
region_id

[('Камчатский край', 151233),
 ('Мурманская область', 2099216),
 ('Пермский край', 115135),
 ('Свердловская область', 79379),
 ('Вологодская область', 115106),
 ('Владимирская область', 72197),
 ('Калининградская область', 103906),
 ('Челябинская область', 77687),
 ('Новгородская область', 89331),
 ('Республика Крым', 3795586),
 ('Севастополь', 3788485),
 ('Севастополь', 1574364),
 ('Автономна Республіка Крим', 72639),
 ('Ингушетия', 253252),
 ('Тверская область', 2095259),
 ('Сахалинская область', 394235),
 ('Калужская область', 81995),
 ('Самарская область', 72194),
 ('Ивановская область', 85617),
 ('Орловская область', 72224),
 ('Смоленская область', 81996),
 ('Тульская область', 81993),
 ('Чукотский автономный округ', 151231),
 ('Томская область', 140295),
 ('Приморский край', 151225),
 ('Республика Карелия', 393980),
 ('Архангельская область', 140337),
 ('Мордовия', 72196),
 ('Ульяновская область', 72192),
 ('Волгоградская область', 77665),
 ('Астраханская область', 112819),
 ('Ку

In [ ]:
region_id_df = pd.DataFrame(region_id)

In [ ]:
region_id_df.columns = ['region', 'region_id']
region_id_df

,region,region_id
0,Камчатский край,151233
1,Мурманская область,2099216
2,Пермский край,115135
3,Свердловская область,79379
4,Вологодская область,115106
...,...,...
82,Красноярский край,190090
83,Бурятия,145729
84,Иркутская область,145454
85,Кабардино-Балкария,109879


У нас имеется заранее подготовленная таблица с поступающей к нам информацией о кол-ве продаж кофе (три определенные марки, взятые для анализа) за последний месяц по магазинам (есть shop_id, координаты, адрес, название магазина). 

Предоставить сам файл shop.csv не представляется возможным в связи с политикой конфеденциальности компании.

In [ ]:
shop = pd.read_csv('shop.csv')
shop.head()

,shop_id,st_x,st_y,address,title,count_sales
0,26775268,82.958641,55.042546,"ул. Кошурникова, 9, Новосибирск, Новосибирская...","АО ""Тандер""",1
1,27428909,30.315868,59.939095,Санкт-Петербург и область,Перекрёсток Впрок Санкт-Петербург,271
2,25371835,60.367462,54.982973,"ул. Мира, 2, Чебаркуль, Челябинская обл., Росс...",Агроторг ООО,1
3,23845460,37.542359,55.373815,"Театральная ул., 6, Климовск, Московская обл.,...",Агроторг ООО,1
4,24663144,37.358563,54.901168,"ул. Спортивная, 21, Большевик, Московская обл....",X589 11622-Пятерочка,1


Определим функцию, которая по имеющимся у нас координатам магазина будет возвращать название региона (работает через api yandex)

In [ ]:
apikey = your_yandex_apikey
def fetch_region(apikey, coordinates):
    base_url = "https://geocode-maps.yandex.ru/1.x"
    params = {"geocode": coordinates, "apikey": apikey, "format": "json"}
    response = requests.get(base_url, params=params)
        
    if len(response.json()["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"]) >= 3:
        region = response.json()["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["Address"]["Components"][2]["name"]
    else: 
        region = response.json()["response"]["GeoObjectCollection"]["featureMember"][0]["GeoObject"]["metaDataProperty"]["GeocoderMetaData"]["AddressDetails"]["Country"]["AddressLine"]
    
    return region

In [ ]:
region_shop = []
for i in range(shop.shape[0]):
    coord = str(shop.iloc[i][1]) + ', ' + str(shop.iloc[i][2])
    region_shop.append(fetch_region(apikey, coord))
    time.sleep(1.1)

In [ ]:
len(region_shop)

1241

Добавим столбец с названием региона

In [ ]:
shop['region'] = region_shop

In [ ]:
shop

,shop_id,st_x,st_y,address,title,count_sales,region
0,26775268,82.958641,55.042546,"ул. Кошурникова, 9, Новосибирск, Новосибирская...","АО ""Тандер""",1,Новосибирская область
1,27428909,30.315868,59.939095,Санкт-Петербург и область,Перекрёсток Впрок Санкт-Петербург,271,Санкт-Петербург
2,25371835,60.367462,54.982973,"ул. Мира, 2, Чебаркуль, Челябинская обл., Росс...",Агроторг ООО,1,Челябинская область
3,23845460,37.542359,55.373815,"Театральная ул., 6, Климовск, Московская обл.,...",Агроторг ООО,1,Московская область
4,24663144,37.358563,54.901168,"ул. Спортивная, 21, Большевик, Московская обл....",X589 11622-Пятерочка,1,Московская область
...,...,...,...,...,...,...,...
1236,23845627,36.892797,55.905026,"ул. Московская, 61, Истра, Московская обл., Ро...","ООО ""Лента""",1,Московская область
1237,24661823,36.661902,55.175654,"ул. 50 лет Октября, 14а, Балабаново, Калужская...","ООО ""АТАК""",1,Калужская область
1238,24795103,42.199131,47.528986,"пр. Курчатова, 55, эт.1, Волгодонск, Ростовска...",Н523 11706-ПЯТЕРОЧКА,1,Ростовская область
1239,25986738,53.186367,56.880165,"Школьная ул., 58а, Ижевск, республика Удмуртия...",E705 14659-Пятёрочка,2,Удмуртская Республика


In [ ]:
our_region_id_df = shop.groupby('region')['count_sales'].count().sort_values(ascending=False).reset_index()
our_region_id_df

,region,count_sales
0,Москва,135
1,Московская область,109
2,Санкт-Петербург,90
3,Тюменская область,65
4,Свердловская область,50
...,...,...
67,Республика Бурятия,1
68,Курганская область,1
69,Приморский край,1
70,Амурская область,1


Подготовим таблицу, необходимую для отрисовки карты. В ней должна содержать инф-ия о номере региона (полученном на первом шаге из файла admin_level_4.geojson) и кол-ве продаж в данном регионе.

In [ ]:
DF = our_region_id_df.merge(region_id_df, on='region', how='left')
DF = DF[['region_id', 'count_sales', 'region']]

In [6]:
DF

,region_id,count_sales,region
0,102269,135,Москва
1,51490,109,Московская область
2,337422,90,Санкт-Петербург
3,140291,65,Тюменская область
4,79379,50,Свердловская область
...,...,...,...
82,151234,1,Республика Саха (Якутия)
83,151228,1,Магаданская область
84,151231,1,Чукотский автономный округ
85,151233,1,Камчатский край


Создадим основной объект - карту, указав ее центр (в нашем случаем в центре будет РФ)

In [ ]:
m = folium.Map(location=[63.391522, 96.328125], zoom_start=3)

Добавим к карте регионы РФ с заполнением цветом в соответствии с кол-ом продаж по регионам – хороплет

In [ ]:
rel_ = folium.Choropleth(
       geo_data = data, 
       name = 'Популярность товаров',
       data = DF,
       columns=['region_id', 'count_sales'], 
       key_on='id',
       bins = 5,
       fill_color='OrRd',
       nan_fill_color='yellow',
       nan_fill_opacity=0.05,
       fill_opacity=1,
       line_opacity=0.5,
       legend_name= 'Кол-во продаж по регионам',
       highlight = True,
       show = False,
)

In [ ]:
# При необходимости легенду можно удалить следующим образом
for key in rel_._children:
    if key.startswith('color_map'):
        del(rel_._children[key])

Добавим хороплет к карте

In [ ]:
rel_.add_to(m)

Сохраним карту

In [ ]:
m.save("map.html")